In [1]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
import pyarrow as pa
from datetime import datetime
import json
from time import sleep
import csv
import re
import os.path as osp
import os
import shutil

# HDFS
host = 'localhost'
hdfs_port = 8020
hdfs_username = 'hdfs'
hdfs_password = 'naya'
hdfs_driver = 'libhdfs'
hdfs_kerberos_ticket = None
hdfs_extra_conf = None
stg_dir = '/tmp/project/'

# # Topics/Brokers
topic1 = 'TweeterArchive'
brokers = ['localhost:9092']


In [2]:
consumer = KafkaConsumer(
    topic1,
    group_id = 'File_MySQL_HDFS',
    bootstrap_servers = ['localhost:9092'],
    auto_offset_reset = 'largest',
    enable_auto_commit = True,
    auto_commit_interval_ms = 1000)

In [3]:
fs = pa.hdfs.connect(
    host='localhost', 
    port=8020, 
    user='hdfs', 
    kerb_ticket=None, 
    driver='libhdfs', 
    extra_conf=None)

In [4]:
#create hdsf staging dir if needed

if fs.exists(stg_dir):
    fs.rm(stg_dir, recursive=True)
    fs.mkdir(stg_dir)
else:
    fs.mkdir(stg_dir)

In [5]:
#create local staging dir if needed

if  osp.exists(stg_dir):
    shutil.rmtree(stg_dir, ignore_errors=True)
    os.mkdir(stg_dir)
else:
    os.mkdir(stg_dir)

In [ ]:
for i, message in enumerate(consumer):
    events = json.loads(message.value)
    event_ts = 'event_' + datetime.now().strftime("%Y_%m_%d_%H-%M-%S") + '.json'  
    print(events) 
    file_path = '/tmp/project/' + event_ts
    with open(file_path,'w') as f:
        json.dump(events, f)
    with  open(file_path, 'rb') as f:
        fs.upload(file_path, f)  